# Analyzing River Thames Water Levels
Time series data is everywhere, from watching your stock portfolio to monitoring climate change, and even live-tracking as local cases of a virus become a global pandemic. In this project, you’ll work with a time series that tracks the tide levels of the Thames River. You’ll first load the data and inspect it data visually, and then perform calculations on the dataset to generate some summary statistics. You’ll end by reducing the time series to its component attributes and analyzing them. 

The original dataset is available from the British Oceanographic Data Center.

Here's a map of the locations of the tidal meters along the River Thames in London.

![](locations.png)

The provided datasets are in the `data` folder in this workspace. For this project, you will work with one of these files, `10-11_London_Bridge.txt`, which contains comma separated values for water levels in the Thames River at the London Bridge. After you've finished the project, you can use your same code to analyze data from the other files (at other spots in the UK where tidal data is collected) if you'd like. 

The TXT file contains data for three variables, described in the table below. 

| Variable Name | Description | Format |
| ------------- | ----------- | ------ |
| Date and time | Date and time of measurement to GMT. Note the tide gauge is accurate to one minute. | dd/mm/yyyy hh:mm:ss |
| Water level | High or low water level measured by tide meter. Tide gauges are accurate to 1 centimetre. | metres (Admiralty Chart Datum (CD), Ordnance Datum Newlyn (ODN or Trinity High Water (THW)) | 
| Flag | High water flag = 1, low water flag = 0 | Categorical (0 or 1) |



In [102]:
# We've imported your first Python package for you, along with a function you will need called IQR
import pandas as pd               

def IQR(column): 
    """ Calculates the interquartile range (IQR) for a given DataFrame column using the quantile method """
    q25, q75 = column.quantile([0.25, 0.75])
    return q75-q25

# Begin coding here ...

In [103]:
# loading the dataset
thames = pd.read_csv("data/10-11_London_Bridge.txt")
thames = thames.iloc[:, :3]

# cleaning the dataset
thames.columns = ["date", "water_level", "flag"]
thames["water_level"] = thames["water_level"].astype("float")
thames["date"] = pd.to_datetime(thames["date"])
thames["year"] = thames["date"].dt.year
thames["month"] = thames["date"].dt.month
thames["day"] = thames["date"].dt.day
thames = thames.dropna()

In [104]:
thames

,date,water_level,flag,year,month,day
0,1911-01-05 15:40:00,3.7130,1,1911,1,5
1,1911-02-05 11:25:00,-2.9415,0,1911,2,5
2,1911-02-05 16:05:00,3.3828,1,1911,2,5
3,1911-03-05 11:50:00,-2.6367,0,1911,3,5
4,1911-03-05 16:55:00,2.9256,1,1911,3,5
...,...,...,...,...,...,...
115498,1995-12-30 20:44:00,3.2900,1,1995,12,30
115499,1995-12-31 02:32:00,-1.6000,0,1995,12,31
115500,1995-12-31 08:59:00,3.2000,1,1995,12,31
115501,1995-12-31 15:03:00,-1.8000,0,1995,12,31


In [105]:
# mean, median, mode, IQR for low tide
thames_low = thames[thames["flag"] == 0]
thames_low_data = thames_low["water_level"].agg(["mean", "median", IQR])
thames_low_data

mean     -2.383737
median   -2.412900
IQR       0.538200
Name: water_level, dtype: float64

In [106]:
# mean, median, mode, IQR for high tide
thames_high = thames[thames["flag"] == 1]
thames_high_data = thames_high["water_level"].agg(["mean", "median", IQR])
thames_high_data

mean      3.318373
median    3.352600
IQR       0.743600
Name: water_level, dtype: float64

In [107]:
# annual percent of very high tide
very_high = thames_high["water_level"].quantile(0.9)
very_high_tide = thames_high[thames_high["water_level"] > very_high]
total_high_tide_days = thames_high.groupby("year")["water_level"].count()
very_high_tide_days = very_high_tide.groupby("year")["water_level"].count()
percent_high_tide = very_high_tide_days/total_high_tide_days
percent_high_tide = percent_high_tide.reset_index()
percent_high_tide

,year,water_level
0,1911,0.004098
1,1912,0.032316
2,1913,0.082212
3,1914,0.055313
4,1915,0.045045
...,...,...
80,1991,0.096317
81,1992,0.103253
82,1993,0.145923
83,1994,0.150355


In [108]:
# annual percent of very low tide
very_low = thames_low["water_level"].quantile(0.1)
very_low_tide = thames_low[thames_low["water_level"] < very_low]
total_low_tide_days = thames_low.groupby("year")["water_level"].count()
very_low_tide_days = very_low_tide.groupby("year")["water_level"].count()
percent_low_tide = very_low_tide_days/total_low_tide_days
percent_low_tide = percent_low_tide.reset_index()
percent_low_tide

,year,water_level
0,1911,0.060606
1,1912,0.066667
2,1913,0.022388
3,1914,0.039017
4,1915,0.033435
...,...,...
80,1991,0.150355
81,1992,0.107496
82,1993,0.112696
83,1994,0.106383


In [109]:
# solution dictionary
solution = {"high_statistics": thames_high_data, "low_statistics": thames_low_data, "very_high_ratio": percent_high_tide, "very_low_ratio": percent_low_tide}